# Modélisations

### Choix sur la base de données

Comme evoqué au début, nous offrons la ici la possibilité de ne garder que les villes de plus de 1000 habitants (aux deux periodes) pour une question de cohérence. En effet, les villes de moins de 1000 habitants peuvent suivre un mode de scrutin différent. Par ailleurs, dans une petite ville, le maire a plus de chance d'être réélu sur des variables non incluses de par sa proximité plus forte avec ses concitoyens.

In [1]:
# Variable booléenne
#Si True, on ne garde que les villes de plus de 1000 habitants
grosse_Ville = True

Dès que cela est possible, nous allons créer une variable valant l'évolution d'évolution entre 2014 et 2019.
Cela va être possible si la variable existe aux deux périodes.

Exemple : Pop_diff = population_2019 - population_2013 (Ces noms de variables sont de circonstances et non les mêmes que dans notre base).

In [2]:
# Prenons-nous en compte l'évolution de 2013 à 2019 ?
# Mettre la variable à True si cela est souhaité
dynamique_temporelle = True

Afin de savoir où en est l'exécution du notebook, il est possible d'activer ce paramètre.

In [40]:
Suivi = True

## 0 - Création de la base d'entraînement et de celle de test

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [5]:
import requests
import zipfile
import io
import pandas as pd
from shutil import rmtree
from jyquickhelper import add_notebook_menu
from unidecode import unidecode
import numpy as np

import sklearn.metrics

from sklearn import svm
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
add_notebook_menu()


In [6]:
insee = pd.read_csv('Données élections et INSEE.csv')
insee = insee.drop("Unnamed: 0", axis = 1)

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [7]:
# On crée la dataframe avec les observations où le maire s'est représenté
insee2 = insee[insee["Tentative de réélection"]==True]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Gestion de la taille des villes considérées :

In [8]:
if grosse_Ville == True :
    
    insee2 = insee2[insee2["P19_POP"] >= 1000]    
    insee2 = insee2[insee2["P13_POP"] >= 1000]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Nous avons besoin du vecteur contenant la variable à prévoir.

In [9]:
# On crée le vecteur contenant résultat
Reelection = insee2["Nom de l'élu en 2020"] == insee2["Nom de l'élu en 2014"]

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [10]:
# On va réduire insee2 en ne gardant que les données de 2014 et 2020 qui sont des valeurs chiffrées
# Valeurs chiffrées
temp = insee2.select_dtypes(include = ["int64", "float64"])
insee2 = pd.concat([insee2["CODGEO"], temp], axis = 1)
# Années gardées
for i in insee2.columns :
    if i[0:3]!="P19" and i[0:3]!="P13" and i!="CODGEO":
        insee2 = insee2.drop(i, axis = 1)
        
# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Si l'on veut être surs que nos modèles soient comparables, il faut centrer et réduire nos données.

In [11]:
temp2 = insee["CODGEO"]

var = insee2.std()
moy = insee2.mean()

insee2 = (insee2-moy)/var
insee2["CODGEO"]=temp2

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


In [12]:
# Certaines de nos colonnes contiennent (énormément) de NaNs.
# Seulement quelques colonnes, redondantes entre 2013 et 2020, sont concernées.
# Par simplicité, nous les enlevons donc.
liste = []
for i in insee2.columns:
        if insee2[i].isnull().values.any()==True :
            liste.append(i)
            
insee2 = insee2.drop(liste, axis = 1)

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Gestion de la prise en compte ou non de la dynamique entre 2013 et 2019 :

In [13]:
if dynamique_temporelle == True :
    
    liste = []
    # On fait la liste des variables qui existent aux deux périodes
    for i in insee2.columns :
        
        if i[0:3] == "P19" :
            temp = "P13" + i[3:]
            
            for j in insee2.columns :
                if j == temp :
                    liste.append(i[3:])
                    break

# On ajoute les variables dynamiques possibles à insee2
for k in liste :
    n1 = "P19" + k
    n2 = "P13" + k
    n3 = k + "_Diff_13_19"
    insee2[n3] = insee2[n1] - insee2[n2]
    
# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


On crée maintenant les bases d'entraînement et de test pour nos différentes modélisations.

In [30]:
X_train, X_test, y_train, y_test = train_test_split(insee2, Reelection, test_size=0.2, random_state=6)

# Suivi de l'exécution si voulu
if Suivi == True :
    print("Tâche effectuée")

Tâche effectuée


Nous allons tester plusieurs méthodes. Afin de savoir laquelle est la "meilleure", nous allons les comparer grâce à quatre indicateurs : l'exactitude (accuracy), la précision, le rappel (recall) ainsi que la statistique F1.

L'exactitude regarde à quel point les données simulées sont proches de la réalité et regarde donc la proportion des résultats dont l'issue est la bonne.

La précision correspond au nombre de vrais positifs sur le nombre de prédictions positives.

Le rappel correspond au nombre de vrais positifs sur le nombre réel de positifs.

La statistique F1 correspond à $\frac{2 \times precision \times rappel}{precision + rappel}$.

In [31]:
# Création de la base contenant ces données :
Stats_Modeles = pd.DataFrame(columns = ["Modèle", "Accuracy", "Precision", "Recall", "Score F1", "Coefficients tous nuls ?", "Nombre de coefficients non nuls"])

# Prend un vecteur et renvoie le nombre d'éléments non nuls de ce vecteur
# eps_ou_non est un booléen.
#S'il vaut True, on ne considère pas une églité stricte, mais si la distance entre deux valurs est supérieure à eps ou non
def nombre_Non_Nuls(vect, eps = 0, eps_ou_non = False):
    
    non_nul = 0
    if eps_ou_non == False :
        for i in range(len(vect)):
            if vect[i] != 0 :
                non_nul = non_nul + 1
                
    else :
        for i in range (len(vect)):
            if (abs(vect[i])) > 0 :
                non_nul = non_nul + 1
            
    return non_nul

# Ajoute une ligne à la dataframe avec les statistiques du modèle
# M correspond au nom du modèle
# y_t correspond à l'échantillon connu 
# y_p correspond à l'échantillon prédis
# C dit si les coefficients sont nuls s'ils existent (ce qui correspond à un modèle constat)
# nb_C donne le nombre de coefficients non nuls s'ils existent
def plusUneLigne(M, y_t, y_p, C, nb_C):
    
    # On évalue les stats du modèle
    ac = sklearn.metrics.accuracy_score(y_t, y_p)
    prec = sklearn.metrics.precision_score(y_t, y_p)
    recall = sklearn.metrics.recall_score(y_t, y_p)
    f1 = sklearn.metrics.f1_score(y_t, y_p)
    

    return Stats_Modeles.append({"Modèle" : M,
                                          "Accuracy" : ac,
                                          "Precision" : prec,
                                          "Recall" : recall,
                                          "Score F1" : f1,
                                          "Coefficients tous nuls ?" : C,
                                          "Nombre de coefficients non nuls" : nb_C},
                                          ignore_index=True)

# Calcule le modèle et l'ajoute à la liste recensant l'efficacité des différents modèles
# clf correspond au modèle utilisé
# nom correspond au nom du modèle
# X_te et y_te aux variables de test
# X_tr, y_tr aux variables d'entraînement
# coeffs est un booléen selon de si le modèle utilise des coefficients
# binarisation est un booléen informant de s'il faut binariser le modèle
def modelise_Puis_Resume(clf, nom, X_tr, y_tr, X_te, y_te, coeffs, binarisation):
    
    # Calcul du modèle
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    
    # Binarisation si nécessaire des modèles autres
    if binarisation == True :
        for j in range(len(y_pred)):
            if y_pred[j]<0.5:
                y_pred[j] = 0
            else :
                y_pred[j] = 1
                
    # Prise en compte des informations des coefficients si le modèle en utilise
    nul = np.nan
    non_nuls = np.nan
    if coeffs == True :
        if clf.coef_.ndim == 1:
            nul = np.array_equal(clf.coef_, np.zeros(X_te.shape[1]))
            non_nuls = nombre_Non_Nuls(clf.coef_)
            
        elif clf.coef_.ndim == 2 and clf.coef_.shape[0] == 1 :
            nul = np.array_equal(clf.coef_[0], np.zeros(X_te.shape[1]))
            non_nuls = nombre_Non_Nuls(clf.coef_[0])
        else :
            print("Modèle pas encore compatible")
 
    S_M = plusUneLigne(nom, y_te, y_pred, nul, non_nuls)
    return S_M
    

## 1 - Méthode du lasso

Nous avons ici une base de données avec beaucoup de variables. De ce fait, avec la méthode du lasso, qui ajoute une pénalisation correspondant à la norme 2 du vecteur des $\beta_i$. La formule du lasso est la suivante :
$ \hat{\beta} = argmin_{\beta \in \mathbb{R}^k } = \frac{1}{n} \sum_{i=1}^{n}{(Y_i - X_i^{'}  \beta)^2 + \lambda ||\beta||_1}$ où $\lambda \in \mathbb{R}^{+}$ est un coefficient de pondération. Plus $\lambda$ est grand, plus le nombre de coefficients $\beta_i$ égal à zéro augmente.

Ce modèle est continue donc on ne trouvera pas un $\hat{y}_i \in \{ 0; 1\}$ mais dans $[\ 0; 1]\ $ correspondant plutôt à une probabilité d'être réélu(e). On binarise alors notre vecteur en donnant la valeur de un aux probabilités supérieures à $\frac{1}{2}$ et zéro à celles inférieures : $ \hat{y_i}' = \mathbb{1}_{ [\ \frac{1}{2}; 1]\ }(y_i) $.

In [38]:
lambdas = [ 0.1, 0.001, 0.0001, 0.00001, 0]

for i in lambdas :
    clf = linear_model.Lasso(alpha=i)
    nom_temp = "Lasso (α = " + str(i)+ ")"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, True, True)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")
    

## 2 - Régression logistique

Le second modèle que nous allons utiliser correspond au modèle logistique.

In [33]:
solvers = ["lbfgs", "liblinear", "sag" ]

for i in solvers:
    clf = LogisticRegression(random_state=0, solver = i, max_iter = 1000)
    nom_temp = "Logistique (solver = " + i + ")"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, True, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  Logistique (solver = lbfgs)  effectué
Modèle  Logistique (solver = liblinear)  effectué
Modèle  Logistique (solver = sag)  effectué


## 3 - Méthode des k plus proches voisins

La méthode des k plus proches voisins ($ k \in \mathbb{N}^* $) regarde l'issue la plus présente parmi les k observations qui sont les plus proches de notre observation donc l'issue est à prévoir. La notion de proche peut varier suivant la distance choisie.

In [34]:
combien_voisins = [5, 11, 101, 1001]   

for i in combien_voisins :    
    clf = KNeighborsClassifier(n_neighbors=i)
    nom_temp = str(i) + " plus proches voisins"
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, False, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  5 plus proches voisins  effectué
Modèle  11 plus proches voisins  effectué
Modèle  101 plus proches voisins  effectué
Modèle  1001 plus proches voisins  effectué


## 4 - Modèle SVM

On essaie également ce modèle vu en cours qui essaie de séparer les données en divers hyperplans.

In [35]:
kernels = ["linear", "rbf", "sigmoid", "poly"]

for i in kernels :

    clf = SVC(kernel = i, degree = 10)
    nom_temp = "SVM (kernel = " + i + ")"
    
    if i == "linear" :
        coeffic = True
    else :
        coeffic = False
    
    
    Stats_Modeles = modelise_Puis_Resume(clf, nom_temp, X_train.drop("CODGEO", axis=1), y_train, X_test.drop("CODGEO", axis=1), y_test, coeffic, False)
    
    # Suivi de l'exécution si voulu
    if Suivi == True :
        print("Modèle ", nom_temp, " effectué")

Modèle  SVM (kernel = linear)  effectué
Modèle  SVM (kernel = rbf)  effectué
Modèle  SVM (kernel = sigmoid)  effectué
Modèle  SVM (kernel = poly)  effectué


In [39]:
Stats_Modeles

,Modèle,Accuracy,Precision,Recall,Score F1,Coefficients tous nuls ?,Nombre de coefficients non nuls
0,Lasso (α = 0.1),0.849711,0.849711,1.000000,0.918750,True,0
1,Lasso (α = 0.001),0.848059,0.849462,0.998056,0.917784,False,57
2,Lasso (α = 0.0001),0.836499,0.850042,0.980564,0.910650,False,363
3,Lasso (α = 1e-05),0.831544,0.851064,0.971817,0.907441,False,819
4,Lasso (α = 0),0.832370,0.851789,0.971817,0.907853,False,988
5,Logistique (solver = lbfgs),0.834847,0.850380,0.977648,0.909584,False,988
6,Logistique (solver = liblinear),0.834847,0.850380,0.977648,0.909584,False,988
7,Logistique (solver = sag),0.846408,0.849213,0.996113,0.916816,False,988
8,5 plus proches voisins,0.833196,0.848945,0.977648,0.908762,NaN,NaN
9,11 plus proches voisins,0.846408,0.849213,0.996113,0.916816,NaN,NaN


## 4 - Analyse

Blabla.